### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"+f"&fq={filter_query}&sort={sort}&fl={field_list}")

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):  # NYT API limits results to 10 per page, 20 pages = 200 results
    

    # create query with a page number
    # API results show 10 articles at a time
    page_query = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(2)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for responses in response["response"]["docs"]:
            reviews_list.append(responses)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page empty, no results found")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Page empty, no results found


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_normalized_df = pd.json_normalize(reviews_list)
reviews_list_normalized_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# reviews_list_normalized_df["title"] = reviews_list_normalized_df["headline.main"].str.extract(r"\u2018(.*?)\u2019.*Review")
reviews_list_normalized_df['title'] = reviews_list_normalized_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
#reviews_list_normalized_df.rename(columns={"headline.main": "title"}, inplace=True)
reviews_list_normalized_df
print(reviews_list_normalized_df['title'])

0                            The Attachment Diaries
1     What’s Love Got to Do With It?’ Probably a Lo
2                              You Can Live Forever
3                         A Tourist’s Guide to Love
4                           Other People’s Children
5                                    One True Loves
6                    The Lost Weekend: A Love Story
7                                A Thousand and One
8                                Your Place or Mine
9                      Love in the Time of Fentanyl
10                             Pamela, a Love Story
11                                 In From the Side
12                                       After Love
13                                         Alcarràs
14                                   Nelly & Nadine
15                          Lady Chatterley’s Lover
16                           The Sound of Christmas
17                                   The Inspection
18                                    Bones and All
19          

In [8]:
print(reviews_list_normalized_df.columns)

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title'],
      dtype='object')


In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_normalized_df["keywords"] = reviews_list_normalized_df["keywords"].apply(extract_keywords)
reviews_list_normalized_df



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Ti

In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_list_normalized_df["title"].squeeze().to_list()
titles_list


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

In [12]:
# titles_list2 = reviews_list_normalized_df["title"].to_list()
# print(titles_list2)

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [15]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


# Loop through the titles
for title in titles_list:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Sleeping for 12 seconds")
        time.sleep(12)


    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string).json()
    #print(response)
    
    # Convert title to string explicitly
    title = str(title)

    # Convert tmdb_key_string to string if it's not already
    tmdb_key_string = str(tmdb_key_string)
    
    
    #display(response)
    display(response)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
try:
        # Check if there are any results in the response
    if response["results"]:
        # Get movie id
        movie_id = response["results"][0]["id"]


        # Make a request for a the full movie details
        movie_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}").json()
        


        # Execute "GET" request with url
        

        # Extract the genre names into a list
        title = movie_response["title"]
        original_title = movie_response["original_title"]
        budget = movie_response["budget"]
        original_language = movie_response["original_language"]
        homepage = movie_response["original_language"]
        overview = movie_response["overview"]
        popularity = movie_response["popularity"]
        runtime = movie_response ["runtime"]
        release_date = movie_response["release_date"]
        revenue = movie_response["revenue"]
        vote_average = movie_response ["vote_average"]
        vote_count = movie_response ["vote_count"]
        genres = [genre["name"] for genre in movie_response["genres"]]
        spoken_languages = [language["english_name"] for language in movie_response["spoken_languages"]]
        production_countries = [country["name"] for country in movie_response["production_countries"]]


        # Extract the spoken_languages' English name into a list
        spoken_languages = [language["english_name"] for language in movie_response["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_response["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            "title": title,
            "original_title": original_title,
            "budget": budget,
            "original_language" : original_language,
            "homepage": homepage,
            "overview": overview,
            "popularity": popularity,
            "runtime": runtime,
            "release_date":release_date,
            "revenue": revenue,
            "vote_average": vote_average,
            "vote_count": vote_count,
            "genre_names": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries,
        }
        tmdb_movies_list.append(movie_data)

        # print(len(tmdb_movies_list))
        # print(tmdb_movies_list[:5])

         # Print out success message
        print(f"Title found: {title}")

    else:
        print(f"No results found for: {title}")

    # Add other relevant details here
except:

        
        # Print out the title that was found
        print(f"Movie not found: {title}")


{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': None,
   'genre_ids': [18, 9648, 53, 27],
   'id': 743040,
   'original_language': 'es',
   'original_title': 'El apego',
   'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
   'popularity': 2.209,
   'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg',
   'release_date': '2021-10-07',
   'title': 'The Attachment Diaries',
   'video': False,
   'vote_average': 3.0,
   'vote_count': 4}],
 'total_pages': 1,
 'total_results': 1}

{'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg',
   'genre_ids': [18, 10749],
   'id': 887580,
   'original_language': 'en',
   'original_title': 'You Can Live Forever',
   'overview': "When Jaime, a gay teenager, is sent to live in a community of Jehovah's Witnesses in small-town Quebec, she falls madly in love with a devout Witness girl. The two embark on an intense affair with consequences that will reshape the rest of their lives.",
   'popularity': 67.146,
   'poster_path': '/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg',
   'release_date': '2023-03-24',
   'title': 'You Can Live Forever',
   'video': False,
   'vote_average': 6.6,
   'vote_count': 31}],
 'total_pages': 1,
 'total_results': 1}

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/z3Qc5a0grcCREAQKqI195P05dVf.jpg',
   'genre_ids': [10749, 35],
   'id': 813726,
   'original_language': 'en',
   'original_title': "A Tourist's Guide to Love",
   'overview': 'After an unexpected break up, a travel executive accepts an assignment to go undercover and learn about the tourist industry in Vietnam. Along the way, she finds adventure and romance with her Vietnamese expat tour guide and they decide to hijack the tour bus in order to explore life and love off the beaten path.',
   'popularity': 14.594,
   'poster_path': '/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg',
   'release_date': '2023-04-21',
   'title': "A Tourist's Guide to Love",
   'video': False,
   'vote_average': 6.3,
   'vote_count': 153}],
 'total_pages': 1,
 'total_results': 1}

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/33UGFiijBDX18NHecj5xz5GkkHt.jpg',
   'genre_ids': [18, 35],
   'id': 803694,
   'original_language': 'fr',
   'original_title': 'Les Enfants des autres',
   'overview': 'Rachel loves her life, her students, her friends, her ex, her guitar lessons. When she falls in love with Ali, she grows close to his 4-year-old daughter, Leila. She tucks her in, looks after her, and loves her like a mother... which she isn’t. Not yet. Rachel is 40. The desire for a family of her own is growing stronger, and the clock is ticking. Is it too late?',
   'popularity': 8.299,
   'poster_path': '/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg',
   'release_date': '2022-09-21',
   'title': "Other People's Children",
   'video': False,
   'vote_average': 6.876,
   'vote_count': 178},
  {'adult': False,
   'backdrop_path': None,
   'genre_ids': [10751],
   'id': 100744,
   'original_language': 'da',
   'original_title': 'Andre folks børn',
   'overview': "The sy

...

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/yd9fYKqbsIXzyATqXKvsTxjEqEb.jpg',
   'genre_ids': [18, 878],
   'id': 776490,
   'original_language': 'pt',
   'original_title': 'A Nuvem Rosa',
   'overview': 'Giovana and Yago are strangers who share a spark after meeting at a party. When a deadly cloud mysteriously takes over their city, they are forced to seek shelter with only each other for company. As months pass and the planet settles into an extended quarantine, their world shrinks, and they are forced to come to terms with an accelerated timeline for their relationship. With all their other interactions governed by screens, and with the strain of isolation setting in, Giovana and Yago struggle to reinvent themselves and reconcile the differences that threaten to tear them apart.',
   'popularity': 2.744,
   'poster_path': '/5bLB5RnwXNxrXji49jcoxWopFnk.jpg',
   'release_date': '2021-04-01',
   'title': 'The Pink Cloud',
   'video': False,
   'vote_average': 6.0,
 

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/fMx8Srs8AsxMr2XXHbFRaH2uMI2.jpg',
   'genre_ids': [18, 10749],
   'id': 501841,
   'original_language': 'en',
   'original_title': 'A Journal for Jordan',
   'overview': 'Based on the true story of First Sergeant Charles Monroe King, a soldier deployed to Iraq begins to keep a journal of love and advice for his infant son. Back at home, senior New York Times editor Dana Canedy revisits the story of her unlikely, life-altering relationship with King and his enduring devotion to her and their child.',
   'popularity': 11.502,
   'poster_path': '/t8xX6kYZj6y0XIxniJdFmaFkvp1.jpg',
   'release_date': '2021-12-22',
   'title': 'A Journal for Jordan',
   'video': False,
   'vote_average': 7.0,
   'vote_count': 103}],
 'total_pages': 1,
 'total_results': 1}

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/tHnHTp50qDx7br1i9ulh74MUW0A.jpg',
   'genre_ids': [18, 10749, 80],
   'id': 511809,
   'original_language': 'en',
   'original_title': 'West Side Story',
   'overview': 'Two youngsters from rival New York City gangs fall in love, but tensions between their respective friends build toward tragedy.',
   'popularity': 17.848,
   'poster_path': '/myAX5qoD6YVLNGiWpk2wcU66Vfq.jpg',
   'release_date': '2021-12-08',
   'title': 'West Side Story',
   'video': False,
   'vote_average': 7.01,
   'vote_count': 1447},
  {'adult': False,
   'backdrop_path': '/rNr3W7GuSpIRjuSr0TM6vSTrIdx.jpg',
   'genre_ids': [80, 18, 10749],
   'id': 1725,
   'original_language': 'en',
   'original_title': 'West Side Story',
   'overview': 'In the slums of the upper West Side of Manhattan, New York, a gang of Polish-American teenagers called the Jets compete with a rival gang of recently immigrated Puerto Ricans, the Sharks, to "own" the neighborhood st

{'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}

No results found for: Aulcie


In [17]:
for title in titles_list:
    print(title)

The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo
You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman
About Fate
Waiting for Bojangles
I Love My Dad
A Love Song
Alone Together
Art of Love
The Wheel
Thor: Love and Thunder
Both Sides of the Blade
Fire of Love
Love & Gelato
Stay Prayed Up
Benediction
Dinner in America
In a New York Minute
Anaïs in Love
I Love America
See You Then
La Mami
Love After Love
Deep Water
Lucy and Desi
Cyrano
The In Between
Book of Love
Lingui, the Sacred Bonds
The Pink Cloud
A Journal for Jordan
West Side Story
Aulcie


In [18]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))



[]


In [19]:
# Convert the results to a DataFrame
convert_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [21]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list_normalized_df, tmdb_movies_list, on="title", how="left")    
merged_df.head()

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = []

# Create a list of characters to remove
columns_to_fix = merged_df.columns

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    if merged_df[column].dtype == 'object':

    # Loop through characters to remove
    for char in chars_to_remove:
        # Replace the character in the column only if it's a string
        merged_df[column] = merged_df[column].apply(lambda x: x.replace(char, "") if isinstance(x, str) else x)
    
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

# Display the fixed DataFrame
merged_df.head()

In [ ]:
print(merged_df.columns)

In [ ]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)
merged_df.head()

In [ ]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("NYT_TMDB.csv", index=False, encoding='utf-8-sig')
merged_df.head()